Assumptions correction

In [ ]:
# Number of points above the mean
m = np.sum(data > mean).values[0]
print('Number of points above the mean, m = %d' % m) 

#using count function
m = data[data > mean].count()
print('Number of points above the mean, m = %d' % m) 

In [ ]:
# Create a new series with the mean subtracted from the original series
new_series = np.array(data - mean).flatten()

# Count how many times the sign changes 
runs = (np.sum(np.diff(np.sign(new_series)) != 0) + 1)
print('Number of runs = %d' % runs) #number of runs

In [ ]:
#Expected number of runs
exp_runs= 2*m*(n-m)/n +1
print('Expected number of runs = %.3f' % exp_runs)

In [ ]:
# Standard deviation of the number of runs
std_runs = np.sqrt((2*m*(n-m)*(2*m*(n-m)-n)/((n**2)*(n-1))))
print('Standard deviation of runs = %.03f' % std_runs)

#95% confidence interval
conf_int= stats.norm.interval(0.95, loc=exp_runs, scale=std_runs)
print('Confidence interval: (%.3f, %.3f)' % (conf_int[0], conf_int[1]))

In [ ]:
# H0: data is random

alpha = 0.05 # significance level
#test statistic
z0 = (runs-exp_runs)/std_runs
z0 = z0.values[0]
print('z0 = %f' % z0)
z_alfa2= stats.norm.ppf(1-alpha/2)
print('z_alfa2 = %f' % z_alfa2)

if abs(z0)>z_alfa2:
  print('The null hypothesis is rejected')
else: 
  print('The null hypothesis is accepted')


# Remember, it is a two-tailed test, so we need to multiply the p-value by 2
p_value = 2 * (1 - stats.norm.cdf(abs(z0)))
print('p-value = %.3f' % p_value)

In [ ]:
# Direct test and pvalue through library

# Import the necessary libraries for the runs test
from statsmodels.sandbox.stats.runs import runstest_1samp

stat, pval_runs = runstest_1samp(data['Ex1'], correction=False)
print('Runs test statistic = {:.3f}'.format(stat))
print('Runs test p-value = {:.3f}'.format(pval_runs))

In [ ]:
#Autocorrelation graph
# acf test
sgt.plot_acf(data['Ex2'], lags = int(len(data)/3), zero=False)
plt.show()

In [ ]:
# Box-Cox transformation

[data_norm, lmbda] = stats.boxcox(data[''])
#data_norm = stats.boxcox(data[''], lmbda = )

print('Lambda = %.3f' % lmbda)

plt.hist(data_norm)
plt.title('Histogram of Box-Cox transformed data')
plt.show()

In [ ]:
#Bartlett's test at lag 1
alpha = 0.05
lag_test = 1
rk = acf_values[lag_test]
z_alpha2 = stats.norm.ppf(1-alpha/2)
print('Test statistic rk = %f' % rk)
print('Rejection region starts at %f' % (z_alpha2/np.sqrt(n)))

if rk>z_alpha2/np.sqrt(n):
    print('The null hypothesis is rejected')
else: print('The null hypothesis is accepted')

In [ ]:
lag_test = 6 # this is just an example; 

# Generally speaking: how many lags?
# Rule of thumb: L<sqrt(n)

Q0_LBQ = lbq[lag_test-1]
print('Q0_LBQ = %f' % Q0_LBQ)

#Rejection region for chi square distribution 
dof = lag_test
chi2_alfa= stats.chi2.ppf(1-alpha,dof)
print('Rejection region starts at %f' % chi2_alfa)

if Q0_LBQ>chi2_alfa:        
  print('The null hypothesis is rejected')                
else: 
  print('The null hypothesis is accepted')

# Compute the p-value for the LBQ test
pval = 1 - stats.chi2.cdf(Q0_LBQ, lag_test)
print('p-value = %f' % pval)

In [ ]:
#LBQ test for autocorrelation
from statsmodels.stats.diagnostic import acorr_ljungbox

lbq_test = acorr_ljungbox(data_norm, lags=[lag_test], return_df=True)
print('LBQ test statistic at lag %d = %f' % (lag_test, lbq_test.loc[lag_test,'lb_stat']))
print('LBQ test p-value at lag %d = %f' % (lag_test, lbq_test.loc[lag_test,'lb_pvalue']))

Gapping

In [ ]:
#Gapping 
gap_size= 6 # this is just an example, you can try different gapping intervals
gap_num= int(len(data)/gap_size)

gap_data= np.zeros((gap_num))
for i in range (gap_num):
    gap_data[i]=data['Ex3'][i*6]

In [ ]:
# built in
# Take one data point every 6
gap_data = data['Ex3'][::gap_size]

Batching

In [ ]:
# Batching
batch_size = 6
batch_num = int(len(data)/batch_size)

j=0
batch_data = np.zeros((batch_num))
for i in range (batch_num):
    batch_data[i]=np.sum(data['Ex3'][j:j+batch_size])/batch_size
    j=j+batch_size

In [ ]:
# Alternative method
# Create a new column in the dataframe with the corresponding batch number
data['Batch'] = np.repeat(np.arange(1, batch_num+1), batch_size)

# Store the batch means in a new dataframe
batch_data = data.groupby('Batch').mean()